## U-Net (2015)
- Contracting Path
    - 입력 이미지의 Context 포착
    - 입력 이미지의 Feature Extrating
    - VGG-Based Architecture
    
- Expanding Path
    - **Upsampling 부분 / 세밀한 Localization을 위한 구성**
    - 정확한 위치를 판별해야 하기 때문에 보다 세부적으로 해상도를 천천히 높여가는 것
    - Coarse Map에서 Dense Prediction을 얻기 위한 구조
    
![unet1](img/unet1.png)

<hr>

## 1. Contracting Path
- 매 Contracting Step마다 3x3 convolution, ReLU, Padding 없음
- 2x2 maxpooling (stride : 2)
- Down-sampling마다 채널의 수를 2배로 늘림

<hr>

## 2. Expanding Path
- Expanding Step마다 2x2 Up-convolution 수행 (천천히 늘림)
- Expanding Step마다 3x3 conv를 두차례씩 반복 (Relu 포함) (Padding 없음)
- Upsampling 마다 Feature map의 크기가 2배로 늘어나며 수는 절반으로 줄어들음
- **Up-conv된 특징맵은 Contracting Path의 테두리가 Cropped된 특징맵과 Concatenation**
- 마지막 Layer에 1x1 conv 연산
- 최종 Segmentation map의 크기는 Input Image 크기보다 작다.

<hr>

## 3. Overlap-Tite Input
- Patch : Image recognition unit, 한번에 많은 부분을 보기 어려우니 이미지를 Patch(타일)로 나눠서 보자
- Input(572x572) => Patch로 뽑아낸건 74x74 => 나머지 padding부분을 mirror padding을 통해 Image를 맞춘 다음 진행
    - 일반적으론 zero-padding을 쓰는데, Mirror Padding 사용 (해당 부분을 대칭으로 채움)
    - **Patch 작업 단위 때문에, 작아진 이미지에 대해 segmentation을 얻기 위해, 이전 영역의 일부분을 포함시키는 것(Copy and Crop을 통한 Concat)**
- Mirror padding 작업시 다음 타일에서 이전 타일의 부분이 겹쳐져있어서 속도적인 부분도 개선 됨.

![unet_mirroring](img/unet_mirroring.jpg)

<hr>

## 4. Data Augmentation
- 3 by 3 elastic 변환 행렬을 사용하여 진행
- Affine Transform (Rotation, Shearing, Translation, Scailing)
- Elastic Distortion (벡터맵의 크기와 방향에 따라서 조절)


![elastic_distortion](img/elastic_distortion.jpg)

<hr>


## U-Net 단점
- 네트워크가 각 패치에 대해 개별적으로 실행되어야 하고 패치가 겹쳐 중복성이 많기 때문에 속도가 느림
- Localization과 Context사이에 trade-off가 존재
    - 큰 사이즈의 patches는 많은 max-pooling을 해야해서 localization의 정확도가 떨어질 수 있음
    - 작은 사이즈의 patches는 협소한 context만 볼 수 있음.
   

## 요약
- Contracting Path과 Expanding Path를 통한 U자형 모형 (Downsampling과 Upsampling)
- 좀 더 정확한 Localization을 위해서 Contracting Path의 Feature를 Copy and Crop해서 Expanding Path와 Concat하여 Upsampling을 한다는 것


<hr>

### 결과
- Stochastic Gradient Descent 사용, batch size를 좀 작게 학습 -> overlap-tile 방식을 통해서 큰 Receptied Field를 확보하는 것이 좋다라는 생각
